<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_vectors_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_vectors_web_lg')


In [ ]:
!mkdir ./results

In [ ]:
!python /content/drive/MyDrive/KAN/train.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/KAN/train.py", line 2, in <module>
    from core.exec import Execution
  File "/content/drive/.shortcut-targets-by-id/1qq5LCjZ3AGmL2Kv20y4s0nEVp9K7G-Mn/KAN/core/exec.py", line 2, in <module>
    from core.model.net import Net
ModuleNotFoundError: No module named 'core.model.net'


In [ ]:
!mkdir ./data
!cp /content/drive/MyDrive/data_path.zip ./data/
!cd ./data && unzip data_path.zip

In [ ]:
!cd ./data && unzip data_path.zip

In [ ]:
# !ls /content/data/data/ques_embeddings/bluebert
!cp -r /content/data/data/ques_embeddings/w2v /content/drive/MyDrive/VQA_path/ques_embeddings

In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy

/content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy


In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo | wc -l

32761


In [ ]:
!ls /content/data/data/ques_embeddings/bioelmo/7563.npy

/content/data/data/ques_embeddings/bioelmo/7563.npy


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)